In [1]:
from collections import Counter
from datetime import datetime
 
import json
 
from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
 
import numpy as np

t1 = datetime.now()
with open("yelp_data/dataset/review.json") as f:
    reviews = f.read().strip().split("\n")
reviews = [json.loads(review) for review in reviews]
print(datetime.now() - t1)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


0:01:35.237912


In [2]:
texts = [review['text'] for review in reviews]

binstars = [0 if review['stars'] <= 3 else 1 for review in reviews]
bin_texts = []
bin_labels = []
limit = 100000  # Change this to grow/shrink the dataset
neg_pos_counts = [0, 0]
for i in range(len(texts)):
    polarity = binstars[i]
    if neg_pos_counts[polarity] < limit:
        bin_texts.append(texts[i])
        bin_labels.append(binstars[i])
        neg_pos_counts[polarity] += 1

In [3]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(bin_texts)
sequences = tokenizer.texts_to_sequences(bin_texts)
data = pad_sequences(sequences, maxlen=300)

In [4]:
model = Sequential()
model.add(Embedding(20000, 128, input_length=300))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'],)
print(len(bin_labels))

200000


In [5]:
history=model.fit(data, np.array(bin_labels), validation_split=0.5, epochs=10)

Train on 100000 samples, validate on 100000 samples
Epoch 1/10
100000/100000 [==============================] - 2822s 28ms/step - loss: 0.3316 - acc: 0.8617 - val_loss: 0.2952 - val_acc: 0.8730
Epoch 2/10
100000/100000 [==============================] - 2838s 28ms/step - loss: 0.2502 - acc: 0.9001 - val_loss: 0.3170 - val_acc: 0.8708
Epoch 3/10
100000/100000 [==============================] - 2887s 29ms/step - loss: 0.2285 - acc: 0.9093 - val_loss: 0.2756 - val_acc: 0.8844
Epoch 4/10
100000/100000 [==============================] - 2854s 29ms/step - loss: 0.2141 - acc: 0.9154 - val_loss: 0.2878 - val_acc: 0.8717
Epoch 5/10
100000/100000 [==============================] - 2794s 28ms/step - loss: 0.2039 - acc: 0.9203 - val_loss: 0.2743 - val_acc: 0.8900
Epoch 6/10
100000/100000 [==============================] - 2837s 28ms/step - loss: 0.1928 - acc: 0.9256 - val_loss: 0.2493 - val_acc: 0.8947
Epoch 7/10
100000/100000 [==============================] - 2787s 28ms/step - loss: 0.1855 - acc